# <center>Перенос стиля с помощью архитектуры *Adaptive Instance Normalization*</center>
Сделаем необходимые импорты и инициализируем константы

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import keras
import tensorflow as tf
from net.style_autoencoder import StyleTransfer
IMAGES_FOLDER = '/mnt/s/CV/StyleTransferData/'
CONTENT_FOLDER = os.path.join(IMAGES_FOLDER, 'test2015')
STYLE_FOLDER = os.path.join(IMAGES_FOLDER, 'wikiart')
BATCH_SIZE = 8
EPOCHS = 5
IMG_SIZE = (256, 256)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Напишем функцию для удаления повреждённых файлов (необходимо раскомментировать последние 2 строки ячейки, если появляются проблемы с чтением файлов во время обучения)

In [2]:
def delete_corrupted_imgs(path) -> None:
    def remove(file) -> None:
        print('=='*10)
        os.remove(os.path.join(path, file))
        print(f'Deleted corrupted {file}')
        
    files = os.listdir(path)
    for file in files:
        try:
            img = tf.io.decode_jpeg(tf.io.read_file(os.path.join(path, file)))
            if img is None:
                remove(file)
        except Exception as e:
            remove(file)

# delete_corrupted_imgs(STYLE_FOLDER)
# delete_corrupted_imgs(CONTENT_FOLDER)

Инициализируем 2 датасета: `content_dataset` - для изображений контента и `style_dataset` - для стилей

In [3]:
dataset_config = dict(
    label_mode=None, 
    labels=None,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=None,
    crop_to_aspect_ratio=True
)

content_dataset = keras.utils.image_dataset_from_directory(
    CONTENT_FOLDER,
    **dataset_config
)
style_dataset = keras.utils.image_dataset_from_directory(
    STYLE_FOLDER,
    **dataset_config
)

dataset = tf.data.Dataset.zip((content_dataset, style_dataset)).batch(BATCH_SIZE, drop_remainder=True)

Found 81434 files.
Found 81434 files.


[<b>Ссылка на оригинальную статью <i>AdaIN</i></b>](https://arxiv.org/pdf/1703.06868.pdf)

[<b>Моя реализация данной модели на <i>Keras / Tensorflow</i></b>](net/style_autoencoder.py)

В отличии от оригинального решения, я использовал вес функции потерь стиля $\lambda = 20$ вместо $\lambda = 0.01$, чтобы использовать меньше эпох для обучения и получить "больше стиля" в результирующем изображении

In [4]:
model = StyleTransfer()
optimizer = keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=1e-4,
        decay_steps=10,
        decay_rate=5e-5
    )
)
loss_fn = keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss_fn=loss_fn)
model.summary()

Model: "style_transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ada_in (AdaIN)                  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Encoder)               │ ?                      │     3,505,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │     3,505,219 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,010,947 (26.74 MB)

 Trainable params: 3,505,219 (13.37 MB)

 Non-trainable params: 3,505,728 (13.37 MB)

Обучаем модель

In [5]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            './checkpoint/model_20_style_loss_{epoch}.keras',
            monitor='total_loss',
            save_best_only=False,
            save_freq='epoch'
        )
    ]
)

Epoch 1/5


I0000 00:00:1710173606.162931  159815 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10179/10179 ━━━━━━━━━━━━━━━━━━━━ 972s 93ms/step - content_loss: 471015.2188 - learning_rate: 9.8755e-05 - style_loss: 1417284.3750 - total_loss: 1888297.5000
Epoch 2/5
10179/10179 ━━━━━━━━━━━━━━━━━━━━ 954s 93ms/step - content_loss: 316740.0625 - learning_rate: 9.4029e-05 - style_loss: 539033.7500 - total_loss: 855772.7500
Epoch 3/5
10179/10179 ━━━━━━━━━━━━━━━━━━━━ 974s 95ms/step - content_loss: 284312.5000 - learning_rate: 8.9734e-05 - style_loss: 465085.6875 - total_loss: 749398.1250
Epoch 4/5
10179/10179 ━━━━━━━━━━━━━━━━━━━━ 977s 96ms/step - content_loss: 268049.6562 - learning_rate: 8.5814e-05 - style_loss: 431344.8438 - total_loss: 699394.4375
Epoch 5/5
10179/10179 ━━━━━━━━━━━━━━━━━━━━ 984s 96ms/step - content_loss: 256738.2969 - learning_rate: 8.2223e-05 - style_loss: 407157.5625 - total_loss: 663895.6250


Результаты обучения будут в [<b>другом ноутбуке</b>](InferenceTest.ipynb)